In [ ]:
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature
import earthpy.spatial as es
import geopandas as gpd
import matplotlib.gridspec
import matplotlib.lines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotnine as p9
import pyproj
import rasterio as rio
import scipy.stats
import shapely.geometry
import sklearn.metrics
import xarray as xr
from PIL import Image
from matplotlib_scalebar.scalebar import ScaleBar
from scipy.ndimage import gaussian_filter

In [ ]:
with plt.style.context(("tableau-colorblind10")):
    color_list = plt.rcParams['axes.prop_cycle'].by_key()['color']
    print(color_list)

In [ ]:
def my_highlight_cols(c, col_to_index: dict, cols_to_ignore=None):
    """
        Highlight a certain for column c using the map col_to_index, which maps which lines should be made bold.
    """
    styles = [''] * len(c)
    col_name = c.name[0] if isinstance(c.name, tuple) else c.name
    if cols_to_ignore is not None and col_name in cols_to_ignore:
        return styles

    # make bold the indicated lines
    styles[col_to_index[col_name]] = 'font-weight:bold'
    return styles

In [ ]:
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'cmr10'
plt.rcParams['mathtext.cal'] = 'cmr10'
matplotlib.rcParams["font.family"] = "cmr10"
matplotlib.rcParams["font.size"] = "12"
plt.rcParams['axes.formatter.use_mathtext'] = True

### Paths

In [ ]:
results_root_dir = Path('../data/external/_experiments/s2_alps_plus')
model_root_dir = results_root_dir / 'unet'
model_version = 'version_0'

# main directory for plots
plots_dir = Path(f'../data/external/scratch/plots')
plots_dir.mkdir(parents=True, exist_ok=True)

#### Read the inventory outlines

In [ ]:
outlines_fp = '../data/outlines/paul_et_al_2020/c3s_gi_rgi11_s2_2015_v2.shp'
gl_df = gpd.read_file(outlines_fp)
gl_df['date_inv'] = gl_df.date_inv.apply(pd.to_datetime)
gl_df['area_inv'] = gl_df.area_km2
gl_df['year_inv'] = gl_df.date_inv.apply(lambda d: d.year)

# number of glaciers and total area 
print(f"n = {len(gl_df)}; area = {gl_df.area_inv.sum():.1f} km2")

# compute how many glaciers are above 0.1 km2 and the percentage of their areas
gl_sdf = gl_df[gl_df.area_inv >= 0.1]
print(
    f"n = {len(gl_sdf)};"
    f" area = {gl_sdf.area_inv.sum():.1f} km2"
    f" ({gl_sdf.area_inv.sum() / gl_df.area_inv.sum() * 100:.1f}%)"
)
gl_sdf

### Table with the number of glaciers and their total area separated by year

In [ ]:
table_df = gl_df.groupby('year_inv').agg({'GLACIER_NR': 'count', 'area_inv': 'sum'}).round(1).reset_index()
table_df.GLACIER_NR = table_df.GLACIER_NR.apply(lambda x: f"{x} ({x / len(gl_df) * 100:.1f}\\%)")
table_df.area_inv = table_df.area_inv.apply(lambda x: f"{x} km$^2$ ({x / gl_df.area_inv.sum() * 100:.1f}\\%)")
table_df.columns = ['Year', 'Count', 'Area']

# add a line with the sum
table_df.loc[len(table_df.index)] = ['All', len(gl_df), f"{gl_df.area_inv.sum():.1f} km$^2$"]

display()

out = table_df.style.hide(axis="index").to_latex(
    hrules=True,
    convert_css=True
)
bold_lines = {c: -1 for c in table_df.columns}

out = table_df.style.hide(axis="index").apply(my_highlight_cols, col_to_index=bold_lines, axis=0).to_latex(
    hrules=True,
    convert_css=True,
    column_format=''.join(['c'] * len(table_df.columns))
)

for x in out.splitlines():
    print(f"\t{x}")

# out = out.replace('_', '\\_')
# out = out.replace('font\\-weightbold', 'font-weightbold')
# print(out)

### Table with the data filtering (QC) results


In [ ]:
df_dates_final = pd.read_csv('../data/inv_images_qc/final_dates.csv')
display(df_dates_final)

table_df_totals = {'Step': [], 'Count': [], 'Area': []}

# add the initial stats
table_df_totals['Step'].append('Before any filtering')
table_df_totals['Count'].append(len(gl_df))
table_df_totals['Area'].append(f"{gl_df.area_inv.sum():.1f} km$^2$")

# area filtering
table_df_totals['Step'].append(r'After filtering by area ($\geq$ 0.1 km$^2$)')
table_df_totals['Count'].append(f"{len(gl_sdf)} ({len(gl_sdf) / len(gl_df) * 100:.1f}%)")
table_df_totals['Area'].append(
    f"{gl_sdf.area_inv.sum():.1f} km$^2$ ({gl_sdf.area_inv.sum() / gl_df.area_inv.sum() * 100:.1f}%)")

# after manual quality check
table_df_totals['Step'].append('After manual quality check')
table_df_totals['Count'].append(f"{len(df_dates_final)} ({len(df_dates_final) / len(gl_df) * 100:.1f}%)")
_df = gl_df[gl_df.entry_id.isin(df_dates_final.entry_id)]
table_df_totals['Area'].append(
    f"{_df.area_inv.sum():.1f} km$^2$ ({_df.area_inv.sum() / gl_df.area_inv.sum() * 100:.1f}%)")

table_df_totals = pd.DataFrame(table_df_totals)
display(table_df_totals)

out = table_df_totals.style.hide(axis="index").to_latex(
    hrules=True,
    convert_css=True,
    column_format=''.join(['l'] * len(table_df_totals.columns))
)

out = out.replace('%', '\\%')
for x in out.splitlines():
    print(f"\t{x}")

# out = out.replace('_', '\\_')
# out = out.replace('font\\-weightbold', 'font-weightbold')
# print(out)

### Table with the glacier statistics of the test folds for each cross-validation iteration


In [ ]:
dir_splits = Path('../data/external/wd/s2_alps_plus/cv_split_outlines/')

table_df_totals = {'Subregion': [], 'Lon range': [], '#Glaciers': [], 'Area': []}
for i_split in range(1, 1 + 5):
    fp = dir_splits / f"split_{i_split}" / f"fold_test.shp"
    print(f"Reading the split df from {fp}")
    gdf = gpd.read_file(fp)
    gdf = gdf.to_crs(epsg=4326)
    print(f"split_{i_split}: n = {len(gdf)}; area = {gdf.area_km2.sum():.1f} km2")
    table_df_totals['Subregion'].append(f"$R_{i_split}$")
    # add leading spaces to make the table look better
    s1 = '\\ ' if gdf.total_bounds[0] < 10 else ''
    s2 = '\\ ' if gdf.total_bounds[2] < 10 else ''
    table_df_totals['Lon range'].append(f"{s1}{gdf.total_bounds[0]:.1f}° - {s2}{gdf.total_bounds[2]:.1f}° E")
    table_df_totals['#Glaciers'].append(len(gdf))
    table_df_totals['Area'].append(gdf.area_km2.sum())
    # fp_split = dir_splits / f"split_{i_split}"
table_df_totals = pd.DataFrame(table_df_totals)
print(table_df_totals.sum())

# add the percentages for the number of glaciers and area
table_df_totals['#Glaciers'] = table_df_totals['#Glaciers'].apply(lambda x: f"{x} ({x / len(gl_sdf) * 100:.1f}%)")
table_df_totals['Area'] = table_df_totals['Area'].apply(
    lambda x: f"{x:.1f} km$^2$ ({x / gl_sdf.area_inv.sum() * 100:.1f}%)")

display(table_df_totals)

out = (
    table_df_totals.style.hide(axis='index')
    .to_latex(
        hrules=True,
        convert_css=True,
        column_format=''.join(['c'] * len(table_df_totals.columns)),
    )
)

# escape Latex characters
out = out.replace('%', '\\%')
out = out.replace('#', '\\#')
out = out.replace('_', '\\_').replace(r'R\_', r'R_')

# rename the first column & make it into two lie

for x in out.splitlines():
    print(f"\t{x}")

print()

### Tables with performance metrics
1. by subregion, using standard ML metrics (for the inventory) - median per glacier
2. by year & glacier size (both the inventory and 2023) - median & sum per glacier class

Metrics are obtained by running the script `eval.sh`)

In [ ]:
ds_name = 's2_alps_plus/inv'
ds_fold = 'test'
seed = 'all'
buffer_m = 0  # we don't allow any mistake

table_df = {'subregion': [], 'accuracy': [], 'iou': [], 'precision': [], 'recall': [], 'f1': []}

df_all = []
for i_split in range(1, 5 + 1):
    fp = f'{model_root_dir}/split_{i_split}/seed_{seed}/{model_version}/output/stats_calib/{ds_name}/s_{ds_fold}/stats_calib.csv'
    df = pd.read_csv(fp)

    # dummy columns so it doesn't fail with buffer_m = 0
    df['area_non_g_b0'] = 0
    df['area_non_g_pred_b0'] = 0

    df['i_split'] = i_split

    df_all.append(df)

df_all = pd.concat(df_all)

for i_split in [1, 2, 3, 4, 5, None]:
    df = df_all.copy()
    if i_split is not None:
        df = df[df.i_split == i_split]

    s = f"b{'-' if buffer_m != 0 else ''}{buffer_m}"
    p = df[f"area_{s}"]
    tp = df[f"area_pred_{s}"]
    fn = p - tp

    # for FP we always look up to 50m
    fp = df['area_non_g_pred_b50'] - df[f"area_non_g_pred_b{buffer_m}"]
    n = df['area_non_g_b50'] - df[f"area_non_g_b{buffer_m}"]
    tn = n - fp

    # compute the 5 metrics
    accuracy = (tp + tn) / (tp + fp + fn + tn) * 100
    iou = tp / (tp + fp + fn) * 100
    precision = tp / (tp + fp) * 100
    recall = tp / (tp + fn) * 100
    f1 = 2 * (precision * recall) / (precision + recall)

    table_df['subregion'].append(rf'$R_{i_split}$' if i_split is not None else 'All')
    for c, v in zip(
            ('accuracy', 'iou', 'precision', 'recall', 'f1'),
            (accuracy, iou, precision, recall, f1)
    ):
        # table_df[c].append(rf"\makecell{{{v.mean():.3f} \\ ± \\ {v.std():.3f}}}")
        prefix = '' if v.std() >= 10 else r'\ '
        table_df[c].append(f"{v.mean():.1f} ± {prefix}{v.std():.1f}")

table_df = pd.DataFrame(table_df)
display(table_df)

out = (
    table_df.style.hide(axis='index')
    .to_latex(
        hrules=True,
        convert_css=True,
        column_format=''.join(['c'] * len(table_df.columns))
    )
)

for x in out.splitlines():
    if 'all' in x:
        print('\t\\midrule')
    print(f"\t{x}")

In [ ]:
# table_df['n_glaciers'].append(len(sdf))
# table_df['area_p'].append(area_p)
# table_df['area_pred_tp'].append(area_pred_tp)
# table_df['area_pred_fn'].append(area_pred_fn)
# table_df['area_n'].append(area_n)
# table_df['area_pred_fp'].append(area_pred_fp)
#
# _df_all = (df.rename(
#     columns={
#         'split': 'Subregion',
#         'BinaryAccuracy': 'Accuracy',
#         'BinaryJaccardIndex': 'IOU',
#         'BinaryPrecision': 'Precision',
#         'BinaryRecall': 'Recall',
#         'BinaryF1Score': 'F1'}
# ).drop(
#     columns=['entry_id', 'Recall_debris', 'loss', 'seed']
# ))
#
# _df_all.subregion = _df_all.subregion.apply(lambda s: s.replace('split', 'R'))

# table_df_avg = _df_all.groupby('subregion').mean(numeric_only=True).reset_index()
# table_df_std = _df_all.groupby(['subregion', 'fp']).std(numeric_only=True).reset_index().drop(
#     columns='fp').groupby(
#     'subregion').agg(lambda x: np.mean(x ** 2) ** 0.5)
#
# display(table_df_avg)
# display(table_df_std)
#
# # make a table showing the avg ± std
# table_df = table_df_avg.copy()
# for c in table_df_std:
#     table_df[c] = [
#         # rf"\makecell{{{v_mean:.3f} \\ ± \\ {v_std:.4f}}}"
#         f"{v_mean:.3f} ± {v_std:.3f}"
#         for (v_mean, v_std) in zip(table_df_avg[c].values, table_df_std[c].values)
#     ]
#
# out = (
#     table_df.style.hide(axis='index')
#     .to_latex(
#         hrules=True,
#         convert_css=True,
#         column_format=''.join(['c'] * len(table_df.columns))
#     )
# )
# out = re.sub(r'(R_\d+)', r'$\1$', out)  # covert to subscript
# out = out.replace('#', '\\#')
#
# # rename the subregion column to make it narrower
# out = out.replace('subregion', 'R')
# out = out.replace('JaccardIndex', 'IOU')
#
# for x in out.splitlines():
#     print(f"\t{x}")

In [ ]:
ds_name = 's2_alps_plus'
model_name = 'unet'
model_v = 'version_0'  # all inputs
subdir = 'inv'
# subdir = '2023'

buffer_m = 0
print(f"buffer_m = {buffer_m}")

fp_results = results_root_dir / model_name / 'stats_all_splits' / f'df_stats_calib_all_{ds_name}_{subdir}_{model_v}_ensemble.csv'
print(f"Reading the processed results from {fp_results}")
df = pd.read_csv(fp_results)

area_thrs = [0.1, 0.2, 0.5, 1, 2, 5, 10, 20, np.inf]  # based on Paul et al. 2020

table_df = {k: [] for k in ['area_class', 'n_glaciers', 'P', 'TP', 'TPR', 'FN', 'FNR', 'N', 'FP', 'FPR', 'rel_unc']}

for i in range(len(area_thrs)):
    if i < len(area_thrs) - 1:
        min_area = area_thrs[i]
        max_area = area_thrs[i + 1]
        area_class = f"[{min_area}, {max_area})" if max_area != np.inf else f">= {min_area}"
    else:
        # use all of them
        min_area = 0
        max_area = np.inf
        area_class = 'All'

    idx = (min_area <= df.area_inv) & (df.area_inv < max_area)
    sdf = df[idx]

    # prepare all the numbers per glacier which will then be combined into avg & totals
    s = f"b{'-' if buffer_m != 0 else ''}{buffer_m}"
    p_all = sdf[f"area_{s}"]
    tp_all = sdf[f"area_pred_{s}"]
    fn_all = p_all - tp_all

    # for FP we always look up to 50m
    n_all = (sdf['area_non_g_b50'] - sdf[f"area_non_g_b{buffer_m}"])
    fp_all = (sdf['area_non_g_pred_b50'] - sdf[f"area_non_g_pred_b{buffer_m}"])

    table_df['area_class'].append(area_class)
    table_df['n_glaciers'].append(len(sdf))
    table_df['P'].append(p_all.sum())
    table_df['TP'].append(tp_all.sum())
    table_df['TPR'].append(tp_all.sum() / p_all.sum() * 100)
    table_df['FN'].append(fn_all.sum())
    table_df['FNR'].append(fn_all.sum() / p_all.sum() * 100)
    table_df['N'].append(n_all.sum())
    table_df['FP'].append(fp_all.sum())
    table_df['FPR'].append(fp_all.sum() / n_all.sum() * 100)
    table_df['rel_unc'].append((fn_all.sum() + fp_all.sum()) / p_all.sum() * 100)
    # table_df['rel_unc'].append((fn_all.sum() + fp_all.sum()) / tp_all.sum() * 100) # v2

    # table_df['area_class'].append(' ')
    # table_df['n_glaciers'].append(' ')
    # table_df['P'].append(p_all.mean())
    # table_df['TP'].append(tp_all.mean())
    # table_df['TPR'].append((tp_all / p_all).mean() * 100)
    # table_df['FN'].append(fn_all.mean())
    # table_df['FNR'].append((fn_all / p_all).mean() * 100)
    # table_df['N'].append(n_all.mean())
    # table_df['FP'].append(fp_all.mean())
    # table_df['FPR'].append((fp_all / n_all).mean() * 100)
    # table_df['rel_unc'].append(((fn_all + fp_all) / tp_all).median() * 100)

# table_df_s = table_df.copy()
# for c in table_df.columns[2:]:
#     v_s = []
#     for i in range(len(table_df)):
#         v_s.append(f"{table_df[c][i].round(i%2 + 1)}")
#     table_df_s[c] = v_s
# table_df_s

table_df = pd.DataFrame(table_df)
table_df['area_class'] = table_df['area_class'].apply(lambda s: f"${s}$".replace(r'>=', r'\geq') if s != ' ' else ' ')

nc = len(table_df.columns)
format_dict = {
    c: v for c, v in zip(
        table_df.columns,
        ['{}', '{:3.0f}'] + [f"{{:{len(str(int(table_df[c].iloc[:-1].max().round()))) + 2}.1f}}" for c in
                             table_df.columns[2:]]
    )}
print([f"{{:{len(str(int(table_df[c].max().round()))) + 2}.1f}}" for c in table_df.columns[2:]])
out = (
    table_df.style.format(format_dict).hide(axis='index')
    .to_latex(
        hrules=True,
        convert_css=True,
        # column_format='cc' + 'r' * (nc - 2),
        column_format='c' * nc,
    )
)
out = out.replace('  ', ' \\ ')

for x in out.splitlines():
    if 'All' in x:
        print('\t\\midrule')
    print(f"\t{x}")

### Table with the regional-level results for the best model vs. 1) the one without dh/dt and 2) the band-ratio models

In [ ]:
df_all_models = []
for model_name, model_v in zip(
        ('unet', 'unet', 'band_ratio_regional', 'band_ratio_glacier-wide'),
        ('version_0', 'version_1', 'version_0', 'version_0')
):
    label = 'ensemble' if model_name == 'unet' else 'individual'
    stats_name = 'stats_calib' if model_name == 'unet' else 'stats'
    fp_results_inv = results_root_dir / model_name / 'stats_all_splits' / f'df_{stats_name}_all_s2_alps_plus_inv_{model_v}_{label}.csv'
    fp_results_2023 = results_root_dir / model_name / 'stats_all_splits' / f'df_{stats_name}_all_s2_alps_plus_2023_{model_v}_{label}.csv'
    print(f"Reading the processed results from\n\t1. {fp_results_inv}\n\t2. {fp_results_2023}")
    df_crt_model = pd.concat([pd.read_csv(fp_results_inv), pd.read_csv(fp_results_2023)])
    df_crt_model['model'] = f"{model_name}_{model_v}"
    df_all_models.append(df_crt_model)

df_all_models = pd.concat(df_all_models)

# check if each glacier has the exactly four predictions for each year
assert (df_all_models.groupby('entry_id').size() == 4 * 2).all()
df_all_models

In [ ]:
# compute the recall and the FP rate using the total predicted areas (so not per glacier)
# use the 20m buffer (i.e. use the area_pred column which should already include the buffer)
assert np.isclose(df_all_models.area_pred, df_all_models.area_pred_b20).all()

df_stats = df_all_models[df_all_models.is_inv_year].groupby('model', sort=False)[
    ['area_inv', 'area_pred_b0', 'area_pred', 'area_non_g_b20_50', 'area_non_g_pred_b20_50']].sum()
display(df_stats)
print(f"#glaciers = {len(set(df_all_models.entry_id))}")
print(f"total area inv = {df_stats.area_inv.iloc[0]:.1f} km2")
print(f"total area b20_50 = {df_stats.area_non_g_b20_50.iloc[0]:.1f} km2")
df_stats['recall'] = df_stats.area_pred_b0 / df_stats.area_inv * 100
df_stats['fp_rate_b20_50'] = df_stats.area_non_g_pred_b20_50 / df_stats.area_non_g_b20_50 * 100

# add the debris recall for the glaciers in 'CH' & at least 1% debris
idx = df_all_models.is_inv_year & (df_all_models.Country == 'CH') & (df_all_models.area_debris_p >= 0.01)
df_stats_debris_ch = df_all_models[idx].groupby('model', sort=False)[
    ['area_inv', 'area_debris', 'area_debris_pred_b0']].sum()
print(f"total area inv (CH & with debris) = {df_stats_debris_ch.area_inv.iloc[0]:.2f} km2")
print(f"total area debris (CH & with debris) = {df_stats_debris_ch.area_debris.iloc[0]:.2f} km2")
df_stats['recall_debris'] = df_stats_debris_ch.area_debris_pred_b0 / df_stats_debris_ch.area_debris * 100

# create another index level for the columns & keep only the relevant columns
df_stats = pd.DataFrame({('inventory', c): df_stats[c] for c in ['recall', 'recall_debris', 'fp_rate_b20_50']})

# add the total predicted area for the inventory year and then 2023)
df_stats[('inventory', 'total_area_pred')] = df_all_models[df_all_models.is_inv_year].groupby('model',
                                                                                              sort=False).area_pred.sum()
df_stats[('2023', 'total_area_pred')] = df_all_models[~df_all_models.is_inv_year].groupby('model',
                                                                                          sort=False).area_pred.sum()

# compute the rate of change per year
df_area_diffs_per_year = pd.DataFrame({
    'model': df_all_models[df_all_models.is_inv_year].model.values,
    'area_pred_t0': df_all_models[df_all_models.is_inv_year].area_pred.values,
    'area_pred_t1': df_all_models[~df_all_models.is_inv_year].area_pred.values,
    'num_years_diff': df_all_models[~df_all_models.is_inv_year].year.values - df_all_models[
        df_all_models.is_inv_year].year.values
})
df_area_diffs_per_year['area_rates_per_year'] = \
    (df_area_diffs_per_year.area_pred_t1 - df_area_diffs_per_year.area_pred_t0) / df_area_diffs_per_year.num_years_diff
df_area_diffs_per_year_stats = df_area_diffs_per_year.groupby('model', sort=False).sum()
df_area_diffs_per_year_stats['area_rates_per_year_prc'] = \
    df_area_diffs_per_year_stats.area_rates_per_year / df_area_diffs_per_year_stats.area_pred_t0 * 100
# overall_change_rate_per_year = area_diffs_per_year.sum() / area_pred_t0.sum()
# overall_change_rate_per_year
df_stats[('', 'area_rates_per_year_prc')] = df_area_diffs_per_year_stats['area_rates_per_year_prc']

df_stats
# df_stats['total_area_pred']

In [ ]:
# Note: the columns will be renamed in Latex; copy only the values
table_df_totals = df_stats.copy()

# switch the third column with the first one
cols = table_df_totals.columns.tolist()
table_df_totals = table_df_totals.reindex(columns=[cols[3]] + cols[:3] + cols[4:])

# add the method names in the first column
table_df_totals.insert(loc=0, column='method',
                       value=['DL4GAM', 'DL4GAM (no dh/dt)', 'band-ratio (v1)', 'band-ratio (v2)'])
display(table_df_totals)

format_dict = {c: v for c, v in
               zip(table_df_totals.columns, ['{}', '{:.1f}', '{:.1f}', '{:.1f}', '{:.1f}', '{:.1f}', '{:.2f}'])}
out = (
    table_df_totals.style.format(format_dict).hide(axis='index')
    .to_latex(
        hrules=True,
        convert_css=True,
        column_format=''.join(['c'] * len(table_df_totals.columns)),
    )
)

for x in out.splitlines():
    # add a line in between the methods
    if x.startswith('band-ratio (v1)'):
        print("\t\\midrule")
    if x.startswith('DL4GAM (no dh/dt)'):
        print("\t\\midrule")
    print(f"\t{x}")

print()

## Figures

### For one glacier plot: 1) the ensemble average 2) the ensemble std and 3) the image with the prediction & the buffer on top

In [ ]:
gl_df_pred_avg = gpd.read_file(
    model_root_dir / 'gdf_all_splits/seed_all/version_0/s2_alps_plus/inv/inv_preds_calib.shp')
gl_df_pred_lb = gpd.read_file(
    model_root_dir / 'gdf_all_splits/seed_all/version_0/s2_alps_plus/inv/inv_preds_calib_low.shp')
gl_df_pred_ub = gpd.read_file(
    model_root_dir / 'gdf_all_splits/seed_all/version_0/s2_alps_plus/inv/inv_preds_calib_high.shp')

In [ ]:
entry_id_to_fp_pred = {
    'g_2164': model_root_dir / 'split_2/seed_all/version_0/output/preds_calib/s2_alps_plus/inv/s_test/g_2164/20160906.nc',
    'g_4165': model_root_dir / 'split_1/seed_all/version_0/output/preds_calib/s2_alps_plus/inv/s_test/g_4165/20150826.nc',
    'g_0208': model_root_dir / 'split_5/seed_all/version_0/output/preds_calib/s2_alps_plus/inv/s_test/g_0208/20150829.nc'
}

entry_id = 'g_2164'
# entry_id = 'g_4165'
# entry_id = 'g_0208'
fp_pred = entry_id_to_fp_pred[entry_id]

# read the raster data
fp_list = list(Path(f'../data/external/wd/s2_alps_plus/inv/glacier_wide/{entry_id}').glob('*.nc'))
assert len(fp_list) == 1
fp = fp_list[0]
nc = xr.open_dataset(fp, decode_coords='all')

# read the ensemble prediction
nc_pred = xr.open_dataset(fp_pred, decode_coords='all')
display(nc_pred)

# add the image data to the prediction dataset
nc_pred['band_data'] = nc.band_data

# keep only a 500m buffer around the glacier from the prediction raster
g_bbox = shapely.geometry.box(*gl_df[gl_df.entry_id == entry_id].iloc[0].geometry.bounds)
g_buff = g_bbox.buffer(500)
g_buff_bbox = shapely.geometry.box(*g_buff.bounds)
nc_pred = nc_pred.rio.clip([g_buff_bbox])
nc_pred

In [ ]:
# project the outlines to the local UTM of the raster
gl_df_proj = gl_df.to_crs(nc.rio.crs)
gl_df_pred_avg_proj = gl_df_pred_avg.to_crs(nc.rio.crs)
gl_df_pred_lb_proj = gl_df_pred_lb.to_crs(nc.rio.crs)
gl_df_pred_ub_proj = gl_df_pred_ub.to_crs(nc.rio.crs)

# increase the font size to compensate for the larger figure size
font_size = 16
original_font_size = matplotlib.rcParams["font.size"]
matplotlib.rcParams["font.size"] = str(font_size)

# dirty hacks to adjust the height ratio of the colorbar and the space between the subplots
is_first = False  # show the titles only for the first glacier
is_last = False  # show the colorbar only for the last glacier
if entry_id == 'g_2164':
    gs = matplotlib.gridspec.GridSpec(1, 4)
    is_first = True
elif entry_id == 'g_4165':
    gs = matplotlib.gridspec.GridSpec(1, 4)
elif entry_id == 'g_0208':
    h_ratio_cb = 0.06
    is_last = True
    gs = matplotlib.gridspec.GridSpec(2, 4, height_ratios=[1, 0.03])
else:
    raise ValueError(f"entry_id = {entry_id}")

# plot the RGB image
fig_w = 18
f = nc_pred.pred.shape[0] / nc_pred.pred.shape[1]
fig_h = fig_w / 3 * f
fig = plt.figure(dpi=250, figsize=(fig_w, fig_h))

img1 = nc_pred.band_data.isel(band=[nc_pred.band_data.long_name.index(b) for b in ['R', 'G', 'B']]).values.copy()
img1 = img1.transpose(1, 2, 0)

# clip and scale the image
vmin = np.quantile(img1, 0.01)
vmax = np.quantile(img1, 0.99)
img1 = np.clip(img1, vmin, vmax)
img1 = (img1 - vmin) / (vmax - vmin)

extent = [nc_pred.x.min(), nc_pred.x.max(), nc_pred.y.min(), nc_pred.y.max()]

### suplot 1 - the RGB image with the glacier outline
ax1 = fig.add_subplot(gs[0, 0])
ax1.imshow(img1, extent=extent)
# plt.grid(alpha=0.1, linestyle='--')
ax1.grid(False)
ax1.xaxis.set_label_position('top')
ax1.xaxis.tick_top()
if is_first:
    ax1.set_xlabel('Easting [m]')
ax1.set_ylabel('Northing [m]')
ax1.yaxis.get_offset_text().set_x(-0.2)

gl_df_proj[gl_df_proj.entry_id == entry_id].plot(
    ax=plt.gca(), facecolor='none', edgecolor=color_list[0], linewidth=1.25, label='inventory'
)
# build the legend (manually; default fails with UserWarning: Legend does not support handles for PatchCollection instances)
if is_first:
    legend_handles = [matplotlib.lines.Line2D([0], [0], color=color_list[0], label=f'inventory')]
    ax1.legend(handles=legend_handles, loc='upper right', prop={'size': font_size - 1})

# add the scalebar
# (https://pypi.org/project/matplotlib-scalebar/: "Set dx to 1.0 if the axes image has already been calibrated by setting its extent")
ax1.add_artist(
    ScaleBar(
        dx=1.0,
        length_fraction=0.2,
        font_properties={'size': font_size - 1},
        frameon=True,
        scale_loc='right',
        location='lower left'
    )
)

### subplot 2 - the ensemble average
ax2 = fig.add_subplot(gs[0, 1])
img = nc_pred.pred.values
im = ax2.imshow(img, cmap='hot', vmin=0, vmax=1, zorder=1)
if is_first:
    ax2.set_title('Ensemble \naverage')
ax2.axis('off')
if is_last:
    cax = fig.add_subplot(gs[1, 1])
    fig.colorbar(im, cax=cax, orientation='horizontal')

### subplot 3 - the ensemble std
ax3 = fig.add_subplot(gs[0, 2])
img = nc_pred.pred_std.values
im = ax3.imshow(img, cmap='viridis', zorder=1, vmin=0, vmax=0.5)
if is_first:
    ax3.set_title('Ensemble \nstddev')
ax3.axis('off')
if is_last:
    cat = fig.add_subplot(gs[1, 2])
    fig.colorbar(im, cax=cat, orientation='horizontal')

# subplot 4 - the prediction and the buffer, on top of the image
ax4 = fig.add_subplot(gs[0, 3])
ax4.imshow(img1, extent=extent)
ax4.axis('off')

# plot the buffer
s_lb = gl_df_pred_lb_proj[gl_df_pred_lb_proj.entry_id == entry_id]
s_ub = gl_df_pred_ub_proj[gl_df_pred_ub_proj.entry_id == entry_id]
s_ub.difference(s_lb).plot(ax=ax4, edgecolor=color_list[7], facecolor=color_list[7], linewidth=0.75)

# plot the predicted glacier outlines and the corresponding uncertainty buffer
s = gl_df_pred_avg_proj[gl_df_pred_avg_proj.entry_id == entry_id]
s.plot(ax=ax4, facecolor='none', edgecolor=color_list[1], linewidth=0.75)

# build the legend (manually; default fails with UserWarning: Legend does not support handles for PatchCollection instances)
if is_first:
    legend_handles = [
        matplotlib.lines.Line2D([0], [0], color=color_list[1], label=f'average pred.'),
        matplotlib.lines.Line2D([0], [0], color=color_list[7], label=f'uncertainty'),
    ]
    # Add the custom legend to the plot
    ax4.legend(handles=legend_handles, loc='upper right', prop={'size': font_size - 1})

# gl_df_pred_lb[gl_df_pred_lb.entry_id == entry_id].plot(ax=plt.gca(), facecolor='none', edgecolor=color_list[1],
#                                                    linewidth=0.75)
# gl_df_pred_ub[gl_df_pred_ub.entry_id == entry_id].plot(ax=plt.gca(), facecolor='none', edgecolor=color_list[2],
#                                                    linewidth=0.75)

area_inv = gl_df_proj[gl_df_proj.entry_id == entry_id].area_inv.values[0]
print(f"area_inv = {area_inv:.2f}")
area_lb = s_lb.area.values[0] / 1e6
area_ub = s_ub.area.values[0] / 1e6
area_avg = s.area.values[0] / 1e6
ax4.set_title(
    f"A$_{{\\mu}}$ = {area_avg:.3f} km$^{{2}}$\n"
    f"68.2% CI = [{area_lb:.3f}, {area_ub:.3f}] km$^{{2}}$",
    fontsize=font_size - 1
)

plt.tight_layout()
pos = ax1.get_position()
fig.subplots_adjust(wspace=0.075)
if is_last:
    fig.subplots_adjust(hspace=-0.2)

# add the North arrow (on the first image only)
if is_first:
    im_north = Image.open(plots_dir / 'north.png')
    f = 0.01 * fig.bbox.xmax / im_north.size[0]
    im_north = im_north.resize((int(f * im_north.size[0]), int(f * im_north.size[1])))
    im_north = np.array(im_north).astype(float) / 255
    im_north[:, :, :3] = 1 - im_north[:, :, :3]  # invert the colors
    pos = ax1.get_position()
    fig.figimage(im_north, xo=pos.x1 * fig.bbox.width * 0.925, yo=pos.y0 * fig.bbox.height * 0.2)
    # for ax in axes:
#     ax.set_aspect('equal')

plt.savefig(plots_dir / f'ensemble_{entry_id}.pdf', bbox_inches='tight')
plt.show()

# reset the font size
matplotlib.rcParams["font.size"] = original_font_size

### Figure with all the inputs and the predictions for a single glacier (will be assembled externally)

In [ ]:
entry_id = 'g_0498'

# read the raster data (for the inventory year and 2023)
fp_list = list(Path(f'../data/external/wd/s2_alps_plus/inv/glacier_wide/{entry_id}').glob('*.nc'))
assert len(fp_list) == 1
fp1 = fp_list[0]

fp_list = list(Path(f'../data/external/wd/s2_alps_plus/2023/glacier_wide/{entry_id}').glob('*.nc'))
assert len(fp_list) == 1
fp2 = fp_list[0]

nc1 = xr.open_dataset(fp1, decode_coords='all')
nc2 = xr.open_dataset(fp2, decode_coords='all')

# read the ensemble predictions
gl_df_pred1 = gpd.read_file(model_root_dir / 'gdf_all_splits/seed_all/version_0/s2_alps_plus/inv/inv_preds_calib.shp')
gl_df_pred2 = gpd.read_file(model_root_dir / 'gdf_all_splits/seed_all/version_0/s2_alps_plus/2023/2023_preds_calib.shp')

# reproject to the local UTM
gl_df_proj = gl_df.to_crs(nc1.rio.crs)
gl_df_pred1 = gl_df_pred1.to_crs(nc1.rio.crs)
gl_df_pred2 = gl_df_pred2.to_crs(nc2.rio.crs)

In [ ]:
# extract the bands needed to compute the NDSI, NDVI and NDWI indices, separately for each year
# NDSI = (Green - SWIR) / (Green + SWIR)
# NDVI = (NIR - Red) / (NIR + Red)
# NDWI = (Green - NIR) / (Green + NIR)
band_data = nc1.band_data.values
swir1 = band_data[nc1.band_data.long_name.index('SWIR')]
r1 = band_data[nc1.band_data.long_name.index('R')]
g1 = band_data[nc1.band_data.long_name.index('G')]
nir1 = band_data[nc1.band_data.long_name.index('NIR')]
y1 = nc1.fn[:4]

band_data = nc2.band_data.values
swir2 = band_data[nc2.band_data.long_name.index('SWIR')]
r2 = band_data[nc2.band_data.long_name.index('R')]
g2 = band_data[nc2.band_data.long_name.index('G')]
nir2 = band_data[nc2.band_data.long_name.index('NIR')]
y2 = nc2.fn[:4]

In [ ]:
for y in (y1, y2):
    for show_outlines in (False, True):
        nc = (nc1 if y == y1 else nc2)

        # plot the RGB image
        fig = plt.figure(dpi=200, figsize=(4, 4))
        img1 = nc.band_data.isel(band=[nc.band_data.long_name.index(b) for b in ['R', 'G', 'B']]).values.copy()
        img1 = img1.transpose(1, 2, 0)

        # clip and scale the image
        vmin = np.quantile(img1, 0.01)
        vmax = np.quantile(img1, 0.99)
        img1 = np.clip(img1, vmin, vmax)
        img1 = (img1 - vmin) / (vmax - vmin)

        extent = [nc.x.min(), nc.x.max(), nc.y.min(), nc.y.max()]
        plt.imshow(img1, extent=extent)
        # plt.grid(alpha=0.1, linestyle='--')
        plt.grid(False)
        plt.xlabel('Easting [m]')
        plt.ylabel('Northing [m]')

        if show_outlines:
            # plot the glacier outlines
            gl_df_proj[gl_df_proj.entry_id == entry_id].plot(
                ax=plt.gca(), facecolor='none', edgecolor=color_list[0], linewidth=2
            )

            # plot the ensemble prediction
            gl_df_pred = gl_df_pred1 if y == y1 else gl_df_pred2
            gl_df_pred[gl_df_pred.entry_id == entry_id].plot(
                ax=plt.gca(), facecolor='none', edgecolor=color_list[1], linewidth=1.5
            )

            # build the legend (manually; default fails with UserWarning: Legend does not support handles for PatchCollection instances)
            legend_handles = [
                matplotlib.lines.Line2D([0], [0], color=color_list[0], label=f'Inventory ({y1})'),
                matplotlib.lines.Line2D([0], [0], color=color_list[1], label=f'Prediction ({y})'),
            ]
            # Add the custom legend to the plot
            plt.legend(handles=legend_handles, loc='upper right')

            # draw a rectangle to mark the snow covered area (for the inventory year)
            if y == y1:
                rect_h, rect_w = 600, 600
                rect_coords = (365400 - rect_w // 2, 5041600 - rect_h // 2, 365400 + rect_w // 2, 5041600 + rect_h // 2)
                rect = plt.Rectangle(
                    xy=tuple(rect_coords[:2]), width=rect_w, height=rect_h, color=color_list[3], fill=False,
                    linewidth=1.5, linestyle='--'
                )
                plt.gca().add_artist(rect)
                img_nc_1_cp = img1.copy()

            # disable the axis labels
            plt.axis('off')

        # add the scalebar & North (for the images without outlines)
        if not show_outlines:
            # add the scalebar
            # (https://pypi.org/project/matplotlib-scalebar/: "Set dx to 1.0 if the axes image has already been calibrated by setting its extent")
            plt.gca().add_artist(
                ScaleBar(
                    dx=1.0,
                    length_fraction=0.2,
                    font_properties={'size': 12},
                    frameon=True,
                    scale_loc='right',
                    location='upper right'
                )
            )

            # # move the y-axis labels to the right
            plt.gca().yaxis.get_offset_text().set_x(-0.2)
            plt.gca().yaxis.get_offset_text().set_size(9)

            # move the x-axis labels to the top
            plt.gca().xaxis.set_label_position('top')
            plt.gca().xaxis.tick_top()

            # add the North arrow
            im_north = Image.open(plots_dir / 'north.png')
            f = 0.075 * fig.bbox.ymax / im_north.size[1]
            im_north = im_north.resize((int(f * im_north.size[0]), int(f * im_north.size[1])))
            im_north = np.array(im_north).astype(float) / 255
            im_north[:, :, :3] = 1 - im_north[:, :, :3]  # invert the colors
            fig.figimage(im_north, xo=fig.bbox.xmax * 0.875, yo=fig.bbox.ymax * 0.055)

        plt.tight_layout()
        plt.savefig(plots_dir / f'img_{y}_outlines_{show_outlines}.png', bbox_inches='tight')
        plt.show()

# print the areas
print(f"Inventory area: {gl_df[gl_df.entry_id == entry_id].area_inv.values[0]:.2f} km2")
area_pred_inv = gl_df_pred1[gl_df_pred1.entry_id == entry_id].area.values[0] / 1e6
print(f"Predicted area (inv): {area_pred_inv:.2f} km2")
area_pred_2023 = gl_df_pred2[gl_df_pred2.entry_id == entry_id].area.values[0] / 1e6
print(f"Predicted area (2023): {area_pred_2023:.2f} km2")
print(f"Annual area change rate: {(area_pred_2023 - area_pred_inv) / area_pred_inv / 8 * 100:.2f}%")

# extract the inset
x_min, y_min, x_max, y_max = rect_coords
shp_mask = shapely.Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)])
mask_rect = ~np.isnan(nc1.rio.clip([shp_mask], drop=False).mask_crt_g.values)

# recompute the normalization statistics using the entire image
img1 = nc1.band_data.isel(band=[nc.band_data.long_name.index(b) for b in ['R', 'G', 'B']]).values.copy()
img1 = img1.transpose(1, 2, 0)

# clip and scale the image
vmin = np.quantile(img1, 0.01)
vmax = np.quantile(img1, 0.99)
img1 = np.clip(img1, vmin, vmax)
img1 = (img1 - vmin) / (vmax - vmin)

nc1_rect = nc1.rio.clip_box(*rect_coords)
img1_rect = nc1_rect.band_data.isel(
    band=[nc1_rect.band_data.long_name.index(b) for b in ['R', 'G', 'B']]).values.copy()
img1_rect = img1_rect.transpose(1, 2, 0)
img1_rect = np.clip(img1_rect, vmin, vmax)
img1_rect = (img1_rect - vmin) / (vmax - vmin)
plt.figure(figsize=(2, 2))
plt.imshow(img1_rect)
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.savefig(plots_dir / 'inset.png')
plt.show()


In [ ]:
# decrease the font size to compensate for the smaller figure size
original_font_size = matplotlib.rcParams["font.size"]
matplotlib.rcParams["font.size"] = "10"

fig_width_in = 10
fig_height_in = 14
cbar_args = {'fraction': 0.04725, 'pad': 0.03}

plt.figure(figsize=(fig_width_in, fig_height_in), dpi=200)
for i in range(1, 25):
    plt.subplot(6, 4, i)
    plt.axis('off')

plt.subplot(6, 4, 1)
img = nc1.dhdt.values
# vmax_abs = np.nanmax(np.abs(img))
vmax_abs = np.nanquantile(np.abs(img), 0.99)
plt.imshow(img, cmap='seismic_r', vmin=-vmax_abs, vmax=+vmax_abs)
plt.title('elevation change\n2010-2014')
cbar = plt.colorbar(**cbar_args)
cbar.set_label('m', labelpad=-10, y=1.1, rotation=0)

plt.subplot(6, 4, 2)
plt.imshow((g1 - swir1) / (g1 + swir1), cmap='viridis')
plt.title(f'NDSI - {y1}')
plt.colorbar(**cbar_args)

plt.subplot(6, 4, 5)
plt.imshow((nir1 - r1) / (nir1 + r1), cmap='viridis')
plt.title(f'NDVI - {y1}')
plt.colorbar(**cbar_args)

plt.subplot(6, 4, 6)
plt.imshow((g1 - nir1) / (g1 + nir1), cmap='viridis')
plt.title(f'NDWI - {y1}')
plt.colorbar(**cbar_args)

plt.subplot(6, 4, 10)
dem = nc1.dem.values
plt.imshow(dem, cmap='terrain')
plt.title('DEM')
cbar = plt.colorbar(**cbar_args)
cbar.set_label('m', labelpad=-10, y=1.1, rotation=0)

plt.subplot(6, 4, 11)
plt.imshow(np.sin(np.deg2rad(nc1.aspect.values)), cmap='jet')
plt.title('sin(aspect)')
plt.colorbar(**cbar_args)

plt.subplot(6, 4, 12)
plt.imshow(np.cos(np.deg2rad(nc1.aspect.values)), cmap='jet')
plt.title('cos(aspect)')
plt.colorbar(**cbar_args)

plt.subplot(6, 4, 13)
plt.imshow(nc1.slope.values, cmap='magma')
plt.title('slope')
cbar = plt.colorbar(**cbar_args)
cbar.set_label('deg', labelpad=-5, y=1.1, rotation=0)

plt.subplot(6, 4, 14)
img = nc1.planform_curvature.values
plt.imshow(img, cmap='magma', vmin=np.quantile(img, 0.005), vmax=np.quantile(img, 0.995))
plt.title('Planform\ncurvature')
cbar = plt.colorbar(**cbar_args)
cbar.set_label(r'm$^{-1}\cdot 100$', labelpad=-5, y=1.1, rotation=0)

plt.subplot(6, 4, 15)
img = nc1.profile_curvature.values
plt.imshow(img, cmap='magma', vmin=np.quantile(img, 0.005), vmax=np.quantile(img, 0.995))
plt.title('Profile\ncurvature')
cbar = plt.colorbar(**cbar_args)
cbar.set_label(r'm$^{-1}\cdot 100$', labelpad=-5, y=1.1, rotation=0)

plt.subplot(6, 4, 16)
plt.imshow(nc1.terrain_ruggedness_index.values, cmap='magma')
plt.title('TRI')
cbar = plt.colorbar(**cbar_args)
cbar.set_label('m', labelpad=-5, y=1.1, rotation=0)

plt.subplot(6, 4, 17)
img = nc2.dhdt.values
# vmax_abs = np.nanmax(np.abs(img))
vmax_abs = np.nanquantile(np.abs(img), 0.99)
plt.imshow(img, cmap='seismic_r', vmin=-vmax_abs, vmax=+vmax_abs)
plt.title('elevation change\n2015-2019')
cbar = plt.colorbar(**cbar_args)
cbar.set_label('m', labelpad=-10, y=1.1, rotation=0)

plt.subplot(6, 4, 18)
plt.imshow((g2 - swir2) / (g2 + swir2), cmap='viridis')
plt.title(f'NDSI - {y2}')
plt.colorbar(**cbar_args)

plt.subplot(6, 4, 21)
plt.imshow((nir2 - r2) / (nir2 + r2), cmap='viridis')
plt.title(f'NDVI - {y2}')
plt.colorbar(**cbar_args)

plt.subplot(6, 4, 22)
plt.imshow((g2 - nir2) / (g2 + nir2), cmap='viridis')
plt.title(f'NDWI - {y2}')
plt.colorbar(**cbar_args)

plt.tight_layout()

# reduce the space between the subplots
plt.subplots_adjust(hspace=0.15, wspace=0.25)

plt.savefig(plots_dir / 'inputs.png', bbox_inches='tight')

plt.show()

# reset the font size
matplotlib.rcParams["font.size"] = original_font_size

### Figure: scatter plot with predicted areas vs. inventory areas

In [ ]:
labels_year = 'inv'
# labels_year = '2023'
if labels_year == '2023':
    fp_results_inv = results_root_dir / 'unet' / 'stats_all_splits' / f'df_stats_calib_agg_s2_alps_plus_manual_2023_{model_version}_ensemble.csv'
    df_glacier_agg = pd.read_csv(fp_results_inv)
else:
    fp_results_inv = results_root_dir / 'unet' / 'stats_all_splits' / f'df_stats_calib_all_s2_alps_plus_inv_{model_version}_ensemble.csv'
    df_glacier_agg = pd.read_csv(fp_results_inv)
    assert len(df_glacier_agg) == df_glacier_agg.is_inv_year.sum()
print(f"fp_results_inv = {fp_results_inv}")
df_glacier_agg

In [ ]:
plot_df = df_glacier_agg.copy()  # only the inventory years
y = plot_df.area_inv
y_pred = plot_df.area_pred_b20
# y_pred = plot_df.area_pred_b0 # only recall

plot_df['rel_area_diff'] = (y_pred - y).abs() / y * 100
n = len(plot_df)
n_better_than_5prc = sum(plot_df.rel_area_diff <= 5)

bias = (y_pred - y).mean()
mae = sklearn.metrics.mean_absolute_error(y, y_pred)

lab = 'inv' if labels_year == 'inv' else 'ref'
yl, xl = f'|$A_{{DL4GAM}}$ - $A_{{{lab}}}$| / $A_{{{lab}}}$ * 100 (%)', f'$A_{{{lab}}}$ (km$^2$)'
print(f"total area difference = {y_pred.sum() - y.sum():.2f} km^2 ({100 * (y_pred.sum() - y.sum()) / y.sum():.2f}%)")
label_df = pd.DataFrame({
    'x': [11],
    'y': [80],
    'desc': f"$A_{{total}}^{{{lab}}}$ = {plot_df.area_inv.sum():.1f} km$^2$"
            f"\n$A_{{total}}^{{DL4GAM}}$ = {plot_df.area_pred.sum():.1f} km$^2$"
            f"\nn = {n}"
            f"\nn$_{{\\leq 5\\%}}$ = {n_better_than_5prc} ({n_better_than_5prc / n * 100:.1f} %)"
            f"\nMAPE = {plot_df.rel_area_diff.mean():.2f} %"
            f"\nMAE = {mae:.2f} km$^2$"
})
p_size = 2.5

# g = (
#         p9.ggplot(plot_df, p9.aes(x='area_inv', y='rel_area_diff'))
#         + p9.geom_hline(yintercept=5, linetype='--', color=color_list[1], size=1)
#         + p9.geom_point(alpha=0.5, stroke=0, size=p_size)
#         + p9.theme(figure_size=(7.5, 3.5), dpi=200, text=p9.element_text(size=10))
#         + p9.scale_x_log10()
#         + p9.labs(title="", x=xl, y=yl)
#         + p9.geom_text(p9.aes(label='desc', x='x', y='y'), data=label_df, size=11, ha='left')
# )
g = (p9.ggplot(plot_df, p9.aes(x='area_inv', y='rel_area_diff')) +
     p9.annotate(
         "text", x=50, y=8, label="5% threshold", color=color_list[1], size=10
     ) +
     p9.geom_point(alpha=0.5, stroke=0, size=p_size) +
     p9.geom_hline(yintercept=5, linetype='--', color=color_list[1], size=1) +
     p9.theme(figure_size=(7.5, 3.5), dpi=200, text=p9.element_text(size=10)) +
     p9.scale_x_log10() +
     p9.ylim([0, 100]) +
     p9.labs(title="", x=xl, y=yl) +
     p9.geom_text(p9.aes(label='desc', x='x', y='y'), data=label_df, size=11, ha='left'))

g.save(plots_dir / f'area_comparison_{labels_year}.pdf')
display(g)

In [ ]:
# sanity check
df_changes = pd.read_csv(
    '../data/external/_experiments/s2_alps_plus/unet/stats_all_splits/df_changes_stats_calib_all_s2_alps_plus_version_0_ensemble.csv')
df_changes = df_changes[df_changes.entry_id.isin(df_t0.entry_id.values)]
df_changes = df_changes.sort_values('entry_id')
print(df_changes.area_inv.sum(), df_t0.area_inv.sum())
print(df_changes.area_t0.sum(), df_t0.area_pred.sum())
print(df_changes.area_t1.sum(), df_t1.area_pred.sum())
df_changes

In [ ]:
df_t0 = pd.read_csv(
    '../data/external/_experiments/s2_alps_plus/unet/stats_all_splits/df_stats_calib_agg_s2_alps_plus_manual_inv_version_0_ensemble.csv')
df_t1 = pd.read_csv(
    '../data/external/_experiments/s2_alps_plus/unet/stats_all_splits/df_stats_calib_agg_s2_alps_plus_manual_2023_version_0_ensemble.csv')
assert len(df_t0) == len(df_t1)

# compute the changes using 1) the 2023 glacier outlines and 2) our predictions
area_changes = df_t1.area_inv.values - df_t0.area_inv.values
area_changes_pred = df_t1.area_pred.values - df_t0.area_pred.values
area_inv_t0 = df_t0.area_inv.values
area_pred_t0 = df_t0.area_pred.values
plot_df = {
    'entry_id': df_t1.entry_id.values,
    'change': area_changes,
    'change_pred': area_changes_pred,
    'change_pred_std': df_changes.area_rate_std.values * 8,
    'rel_change': area_changes / area_inv_t0 * 100,
    'rel_change_pred': area_changes_pred / area_pred_t0 * 100,
    'rel_change_pred_std': df_changes.area_rate_std.values * 8 / area_pred_t0 * 100,
    'area_inv': area_inv_t0,
    'area_inv_colorbar': area_inv_t0,
    'area_inv_pred': df_t0.area_pred.values,
    'area_2023': df_t1.area_inv.values,
    'area_2023_pred': df_t1.area_pred.values,
}
plot_df = pd.DataFrame(plot_df)
plot_df['is_within_unc'] = (plot_df.change_pred - plot_df.change).abs() <= plot_df.change_pred_std
# print(plot_df)

yl = f'($A^{{2023}}_{{ref}} - A^{{2015}}_{{inv}}$) / $A^{{2015}}_{{inv}}$ * 100 (%)'
xl = f'($A^{{\\mu , 2023}}_{{DL4GAM}} - A^{{\\mu , 2015}}_{{DL4GAM}}$) / $A^{{\\mu , 2015}}_{{DL4GAM}}$ * 100 (%)'

corr_s = scipy.stats.spearmanr(plot_df.rel_change_pred, plot_df.rel_change)[0]
plot_df_ok = plot_df[plot_df.is_within_unc]
corr_s_pl = scipy.stats.spearmanr(plot_df_ok.rel_change_pred, plot_df_ok.rel_change)[0]
print(f"corr_s = {corr_s:.3f}")
sep = ''.join(['-'] * 51)
label_df = pd.DataFrame({
    'x': [-33],
    'y': [-78],
    'is_within_unc': [True],
    'desc': f"\nn = {len(plot_df)}"
            f"\n$\\rho_{{Spearman}}$ = {corr_s:.3f}"
            f"\n{sep}"
            f"\n$\\Sigma A^{{2015}}_{{inv}}$ = {plot_df.area_inv.sum():.1f} km$^2$"
            f"\n$\\Sigma A^{{2023}}_{{ref}}$ = {plot_df.area_2023.sum():.1f} km$^2$"
            f"\n$\\Rightarrow \\Delta_{{actual}}$ = {plot_df.area_2023.sum() - plot_df.area_inv.sum():.1f} km$^2$ "
            f"({(plot_df.area_2023.sum() - plot_df.area_inv.sum()) / plot_df.area_inv.sum() / 8 * 100:.2f} % y$^{{-1}}$)"
            f"\n{sep}"
            f"\n$\\Sigma A^{{2015}}_{{DL4GAM}}$ = {plot_df.area_inv_pred.sum():.1f} km$^2$"
            f"\n$\\Sigma A^{{2023}}_{{DL4GAM}}$ = {plot_df.area_2023_pred.sum():.1f} km$^2$"
            f"\n$\\Rightarrow \\Delta_{{DL4GAM}}$ = {plot_df.area_2023_pred.sum() - plot_df.area_inv_pred.sum():.1f} km$^2$ "
            f"({(plot_df.area_2023_pred.sum() - plot_df.area_inv_pred.sum()) / plot_df.area_inv_pred.sum() / 8 * 100:.2f} % y$^{{-1}}$)"
            f"\n{sep}"
            f"\n$1 \\sigma$ coverage: {len(plot_df_ok) / len(plot_df) * 100:.1f} % "
            f"\n(nominal = 68.2%)"
})
p_size = 2.5
max_prc_diff = 10
display(plot_df[plot_df.rel_change_pred > max_prc_diff])
g = (
        p9.ggplot(plot_df, p9.aes(x='rel_change_pred', y='rel_change', color='area_inv', shape='is_within_unc')) +
        # p9.geom_point(alpha=0.7, stroke=0, size=p_size) +
        p9.geom_abline(slope=1, intercept=0, linetype='dashed', color='gray', size=0.8) +
        p9.geom_point(alpha=1.0, size=p_size, stroke=1) +
        p9.scale_shape_manual(values=['x', 'o'], name='Within $1\\sigma$',
                              labels=['No', 'Yes']) +
        p9.scale_color_cmap('viridis', name='$A_{{inv}}$ (km²)', trans='log10',
                            limits=(plot_df.area_inv.min(), 20)) +
        p9.theme(
            figure_size=(7, 6.5), dpi=200,
            text=p9.element_text(size=9),
            legend_key_width=10,
            legend_key_height=100,
        ) +
        p9.xlim([-100, max_prc_diff]) +
        p9.ylim([-100, max_prc_diff]) +
        p9.labs(title="", x=xl, y=yl) +
        p9.geom_text(p9.aes(label='desc', x='x', y='y'), data=label_df, size=8.5, ha='left', color="black") +
        p9.theme(legend_position='right') +
        p9.coord_fixed(ratio=1)
)
g.save(plots_dir / 'change_rates_scatter.pdf')
display(g)

### Figure: scatter plot with predicted uncertainties vs. 1) errors and 2) debris areas (CH only)

In [ ]:
plot_df = df_glacier_agg.copy()  # only the inventory years

# add the country
plot_df = plot_df.merge(gl_df[['entry_id', 'Country']])

y = plot_df.area_inv
y_pred = plot_df.area_pred_b20
# y_pred = plot_df.area_pred_b0 # only recall
plot_df['rel_area_diff'] = (y_pred - y).abs() / y * 100
plot_df['area_pred_std'] = (plot_df.area_pred_high - plot_df.area_pred_low) / 2
plot_df['rel_area_unc'] = plot_df['area_pred_std'] / np.maximum(y_pred, 1e-12) * 100

q = 0.98
# drop the outliers (cause by very low predicted areas)
max_rel_area_unc = plot_df.rel_area_unc.quantile(q)
print(f"max_rel_area_unc = {max_rel_area_unc}")
plot_sdf = plot_df[plot_df.rel_area_unc <= max_rel_area_unc]
print(len(plot_sdf), len(plot_sdf) - len(plot_df))

yl = f'|$A^{{\\mu}}_{{DL4GAM}} - A_{{inv}}$| / $A_{{inv}}$ * 100 (%)'
xl = f'$A^{{\\sigma}}_{{DL4GAM}} / A^{{\\mu}}_{{DL4GAM}}$ * 100 (%)'

corr_s = scipy.stats.spearmanr(plot_sdf.rel_area_unc, plot_sdf.rel_area_diff)[0]
print(f"corr_s = {corr_s:.3f}")
label_df = pd.DataFrame({
    'x': [60],
    'y': [95],
    'desc': f"n = {len(plot_sdf)}"
            f"\n$\\rho_{{Spearman}}$ = {corr_s:.3f}"
})
p_size = 2.5
g = (
        p9.ggplot(plot_sdf, p9.aes(x='rel_area_unc', y='rel_area_diff'))
        + p9.geom_point(alpha=0.5, stroke=0, size=p_size)
        + p9.theme(figure_size=(4, 4), dpi=200, text=p9.element_text(size=10))
        + p9.xlim([0, 100])
        + p9.ylim([0, 100])
        + p9.labs(title="", x=xl, y=yl)
        + p9.geom_text(p9.aes(label='desc', x='x', y='y'), data=label_df, size=11, ha='left')
)
g.save(plots_dir / 'uncertainties_vs_errors.pdf')
display(g)

In [ ]:
# choose only the glaciers from CH & with a debris cover > 1%
plot_df['debris_prc'] = plot_df.area_debris / plot_df.area_inv * 100
plot_sdf = plot_df[(plot_df.Country == 'CH') & (plot_df.debris_prc >= 1)]

q = 0.98
# drop the outliers (cause by very low predicted areas)
n_init = len(plot_sdf)
max_rel_area_unc = plot_df.rel_area_unc.quantile(q)
print(f"max_rel_area_unc = {max_rel_area_unc}")
plot_sdf = plot_sdf[plot_df.rel_area_unc <= max_rel_area_unc]
print(len(plot_sdf), len(plot_sdf) - n_init)

# yl, xl = f'|DL4GAM predicted area - inventory area| (km$^2$)', 'DL4GAM area uncertainty (km$^2$)'

yl = f'Debris coverage (%)'
xl = f'$A^{{\\sigma}}_{{DL4GAM}} / A^{{\\mu}}_{{DL4GAM}}$ * 100 (%)'

corr_s = scipy.stats.spearmanr(plot_sdf.rel_area_unc, plot_sdf.debris_prc)[0]
print(f"corr_s = {corr_s:.3f}")
label_df = pd.DataFrame({
    'x': [60],
    'y': [95],
    'desc': f"n = {len(plot_sdf)}"
            f"\n$\\rho_{{Spearman}}$ = {corr_s:.3f}"
})
p_size = 2.5
g = (
        p9.ggplot(plot_sdf, p9.aes(x='rel_area_unc', y='debris_prc'))
        + p9.geom_point(alpha=0.5, stroke=0, size=p_size)
        + p9.theme(figure_size=(4, 4), dpi=200, text=p9.element_text(size=10))
        + p9.xlim([0, 100])
        + p9.labs(title="", x=xl, y=yl)
        + p9.geom_text(p9.aes(label='desc', x='x', y='y'), data=label_df, size=11, ha='left')
)
g.save(plots_dir / 'uncertainties_vs_debris.pdf')
display(g)

### Figure: SGI2016 comparison

The data is taken from Table S6 (https://www.frontiersin.org/journals/earth-science/articles/10.3389/feart.2021.704189/full#supplementary-material).
1) scatter plot with predicted areas vs. average area across experts
2) scatter plot with predicted uncertainties vs. area standard deviation across experts

In [ ]:
fp = model_root_dir / 'stats_all_splits' / f'df_stats_calib_agg_s2_sgi_inv_{model_version}_ensemble.csv'
print(fp)
df_glacier_agg = pd.read_csv(fp)
df_glacier_agg

In [ ]:
s6_df = pd.read_csv('../data/outlines/sgi2016/table_s6.csv')
s6_df = s6_df[s6_df.entry_id.isin(df_glacier_agg.entry_id)]
print('|'.join([f"{x}" for x in s6_df.entry_id]))
display(s6_df)

tdf = df_glacier_agg[df_glacier_agg.entry_id.isin(s6_df.entry_id)].sort_values('area_inv')[
    ['entry_id', 'area_inv', 'area_pred', 'area_pred_std', 'area_debris', 'area_debris_pred_b20']].merge(
    s6_df.drop(columns=['Glacier name', 'stddev_area_prc']), on='entry_id')
display(tdf)

In [ ]:
plot_df = pd.DataFrame({
    'area_inv': tdf.area_inv,
    'area_inv_std': tdf.stddev_area,
    'area_pred': tdf.area_pred,
    'area_pred_std': tdf.area_pred_std,
})

y = plot_df.area_inv
y_pred = plot_df.area_pred
# y_pred = plot_df.area_pred_b0 # only recall

plot_df['rel_area_diff'] = (y_pred - y).abs() / y * 100
n = len(plot_df)
n_better_than_5prc = sum(plot_df.rel_area_diff <= 5)

bias = (y_pred - y).mean()
mae = sklearn.metrics.mean_absolute_error(y, y_pred)

lab = 'SGI'
yl, xl = f'|$A_{{DL4GAM}}$ - $A_{{{lab}}}$| / $A_{{{lab}}}$ * 100 (%)', f'$A_{{{lab}}}$ (km$^2$)'
print(f"total area difference = {y_pred.sum() - y.sum():.2f} km^2 ({100 * (y_pred.sum() - y.sum()) / y.sum():.2f}%)")
label_df = pd.DataFrame({
    'x': [5],
    'y': [50],
    'desc': f"$A_{{total}}^{{{lab}}}$ = {plot_df.area_inv.sum():.1f} km$^2$"
            f"\n$A_{{total}}^{{DL4GAM}}$ = {plot_df.area_pred.sum():.1f} km$^2$"
            f"\nn = {n}"
            f"\nn$_{{\\leq 5\\%}}$ = {n_better_than_5prc} ({n_better_than_5prc / n * 100:.1f} %)"
            f"\nMAPE = {plot_df.rel_area_diff.mean():.2f} %"
            f"\nMAE = {mae:.2f} km$^2$"
})
p_size = 3.5

# g = (
#         p9.ggplot(plot_df, p9.aes(x='area_inv', y='rel_area_diff'))
#         + p9.geom_hline(yintercept=5, linetype='--', color=color_list[1], size=1)
#         + p9.geom_point(alpha=0.5, stroke=0, size=p_size)
#         + p9.theme(figure_size=(7.5, 3.5), dpi=200, text=p9.element_text(size=10))
#         + p9.scale_x_log10()
#         + p9.labs(title="", x=xl, y=yl)
#         + p9.geom_text(p9.aes(label='desc', x='x', y='y'), data=label_df, size=11, ha='left')
# )
g = (p9.ggplot(plot_df, p9.aes(x='area_inv', y='rel_area_diff')) +
     p9.annotate(
         "text", x=15, y=8, label="5% threshold", color=color_list[1], size=10
     ) +
     p9.geom_point(alpha=0.9, stroke=0, size=p_size) +
     p9.geom_hline(yintercept=5, linetype='--', color=color_list[1], size=1) +
     p9.theme(figure_size=(5.5, 3.5), dpi=200, text=p9.element_text(size=10)) +
     p9.scale_x_log10() +
     p9.ylim([0, 100]) +
     p9.labs(title="", x=xl, y=yl) +
     p9.geom_text(p9.aes(label='desc', x='x', y='y'), data=label_df, size=11, ha='left'))

g.save(plots_dir / f'sgi_area_comparison.pdf')
display(g)

plot_df['rel_area_inv_std'] = plot_df['area_inv_std'] / plot_df['area_inv'] * 100
plot_df['rel_area_unc'] = plot_df['area_pred_std'] / y_pred * 100

yl = f'Relative uncertainty across SGI2016 experts (%)'
xl = f'$A^{{\\sigma}}_{{DL4GAM}} / A^{{\\mu}}_{{DL4GAM}}$ * 100 (%)'

corr_s = scipy.stats.spearmanr(plot_df.rel_area_unc, plot_df.rel_area_inv_std)[0]
print(f"corr_s = {corr_s:.3f}")
label_df = pd.DataFrame({
    'x': [60],
    'y': [95],
    'desc': f"n = {len(plot_df)}"
            f"\n$\\rho_{{Spearman}}$ = {corr_s:.3f}"
})
g = (
        p9.ggplot(plot_df, p9.aes(x='rel_area_unc', y='rel_area_inv_std'))
        + p9.geom_point(alpha=0.9, stroke=0, size=p_size)
        + p9.theme(figure_size=(4, 4), dpi=200, text=p9.element_text(size=10))
        + p9.xlim([0, 25])
        + p9.ylim([0, 25])
        + p9.labs(title="", x=xl, y=yl)
        + p9.geom_abline(linetype='--', color='gray')
    # + p9.geom_text(p9.aes(label='desc', x='x', y='y'), data=label_df, size=11, ha='left')
)
g.save(plots_dir / 'sgi_stddev_comparison.pdf')
display(g)

### Figure & table for the rates interpolation part (for the appendix)

#### Figure with the area change rates for each glacier and the piece-wise linear model

In [ ]:
fp = (
        model_root_dir / 'stats_all_splits' /
        f"df_changes_stats_calib_all_s2_alps_plus_{model_version}_ensemble_extrapolated.csv"
)
print(f"Reading the glacier area change rates dataframe from {fp}")
df_rates_all_g = pd.read_csv(fp)
df_rates_all_g.area_class = df_rates_all_g.area_class.apply(lambda x: x.replace('>=', r'$\geq$'))
area_classes = df_rates_all_g.area_class.unique()
df_rates_all_g['area_class'] = pd.Categorical(df_rates_all_g.area_class, categories=area_classes, ordered=True)

# extract only those glaciers for which direct estimates are available (i.e. not interpolated)
df_rates = df_rates_all_g[~df_rates_all_g.area_rate_orig.isna()]

# sdf = df_rates_all_g[~df_rates_all_g.filtered.isna()]
# sdf[sdf.entry_id.isin(["g_1292", "g_0819", "g_0828", "g_2662", "g_3644", "g_3643"])]

df_rates_all_g

In [ ]:
df = df_rates_all_g[df_rates_all_g.filtered == False]

plt.figure(dpi=200, figsize=(10, 4))
plt.scatter(df.area_inv, df.area_rate_prc * 100, label='DL4GAM estimates', color=color_list[0])
plt.gca().set_xscale('log')
df_rates_all_g = df_rates_all_g.sort_values('area_inv')
plt.plot(df_rates_all_g.area_inv, df_rates_all_g.area_rate_prc_pred * 100, color=color_list[1], linestyle='--',
         linewidth=2.5, label='Quadratic fit')
plt.legend(loc='upper left')

plt.xlabel('Glacier inventory area (km$^2$)')
plt.ylabel(r'Relative area change (% y$^{-1}$)')

# equation_text = rf"$\Delta$A/A = {a:.3f} {'+' if b >= 0 else '-'} {abs(b):.3f} $\cdot$ log$_{{10}}$(A) {'+' if c >= 0 else '-'} {abs(c):.3f} $\cdot$ log$_{{10}}$(A)^2"
# plt.text(0.545, 0.07, equation_text, fontsize=10, transform=plt.gca().transAxes,
#          bbox=dict(facecolor='white', edgecolor='gray', alpha=0.8))

plt.grid()

plt.savefig(plots_dir / 'area_change_rates_extrapolation.pdf', bbox_inches='tight')

plt.show()


In [ ]:
table_df = {'Step': [], 'Count': [], 'area': []}

# add the initial stats
t_n = len(gl_df)
t_a = gl_df.area_inv.sum()
table_df['Step'].append('Inventory')
table_df['Count'].append(f"{t_n} (100%)")
table_df['area'].append(f"{t_a:.1f} km$^2$ (100%)")

# add the stats of the covered glaciers
table_df['Step'].append('Our dataset')
sdf = df_rates_all_g[~df_rates_all_g.filtered.isna()]
table_df['Count'].append(f"{len(sdf)} ({len(sdf) / t_n * 100:.1f}%)")
table_df['area'].append(f"{sdf.area_inv.sum():.1f} km$^2$ ({sdf.area_inv.sum() / t_a * 100:.1f}%)")

# filtering using the uncertainties
sdf = df_rates_all_g[df_rates_all_g.filtered_by_unc == False]
table_df['Step'].append(r'After uncertainty-based filtering (SNR = $\frac{\mu}{\sigma}\leq$ 1)')
table_df['Count'].append(f"{len(sdf)} ({len(sdf) / t_n * 100:.1f}%)")
table_df['area'].append(f"{sdf.area_inv.sum():.1f} km$^2$ ({sdf.area_inv.sum() / t_a * 100:.1f}%)")

# filtering using the uncertainties and recall
sdf = df_rates_all_g[df_rates_all_g.filtered == False]
table_df['Step'].append(r'After filtering by recall ($\geq 90%$)')
table_df['Count'].append(f"{len(sdf)} ({len(sdf) / t_n * 100:.1f}%)")
table_df['area'].append(f"{sdf.area_inv.sum():.1f} km$^2$ ({sdf.area_inv.sum() / t_a * 100:.1f}%)")

table_df = pd.DataFrame(table_df)
display(table_df)

out = table_df.style.hide(axis="index").to_latex(
    hrules=True,
    convert_css=True,
    column_format=''.join(['c'] * len(table_df.columns))
)

out = out.replace('%', '\\%')
for x in out.splitlines():
    print(f"\t{x}")

# out = out.replace('_', '\\_')
# out = out.replace('font\\-weightbold', 'font-weightbold')
# print(out)

In [ ]:
df = df_rates_all_g.copy()

extr_stats = {'n_covered': [], 'n_missing': [], 'area_covered': [], 'area_missing': []}
for i, crt_area_class in enumerate(area_classes):
    idx = (df.area_class == crt_area_class)
    x = df.area_inv[idx].mean()
    idx_ok = idx & (df.filtered == False)
    n = sum(idx)
    n_ok = sum(idx_ok)
    ta = df[idx].area_inv.sum()
    ta_ok = df[idx_ok].area_inv.sum()
    extr_stats['n_covered'].append(n_ok)
    extr_stats['n_missing'].append(n - n_ok)
    extr_stats['area_covered'].append(ta_ok)
    extr_stats['area_missing'].append(ta - ta_ok)
extr_stats_df = pd.DataFrame(extr_stats)
extr_stats_df.insert(0, 'area_class', area_classes)
extr_stats_df.insert(3, 'n', extr_stats_df.n_covered + extr_stats_df.n_missing)
extr_stats_df.insert(len(extr_stats_df.columns), 'area', extr_stats_df.area_covered + extr_stats_df.area_missing)

# add a line with the totals
extr_stats_df.loc[len(extr_stats_df)] = ['all'] + extr_stats_df.sum().tolist()[1:]
extr_stats_df.columns = [
    'Size class',
    r'$n_{covered}$',
    r'$n_{missing}$',
    '$n$',
    r'$A_{covered} (km^2)$',
    r'$A_{missing} (km^2)$',
    '$A (km^2)$'
]
extr_stats_df['Size class'] = extr_stats_df['Size class'].apply(lambda s: f"${s}$")
display(extr_stats_df)

format_dict = {
    c: v for c, v in
    zip(extr_stats_df.columns, ['{}', '{:d}', '{:d}', '{:d}', '{:.2f}', '{:.2f}', '{:.2f}'])
}
out = extr_stats_df.style.format(format_dict).hide(axis="index").to_latex(
    hrules=True,
    convert_css=True,
    column_format=''.join(['c'] * len(extr_stats_df.columns))
)

out = out.replace(r'$\geq$', r'\geq')

for x in out.splitlines():
    if 'all' in x:
        print('\t\\midrule')
    print(f"\t{x}")

### Figures with the glacier extent & with the gridded area change rates, with the hillshade in the background

In [ ]:
gl_df_with_rates_ok = gl_df_with_rates[gl_df_with_rates.filtered == False].copy()
gl_df_with_rates_ok

In [ ]:
matplotlib.rcParams["font.family"] = "serif"  # cmr doesn't work here

In [ ]:
# plot only the glaciers covered in our dataset
gl_df_with_rates = gl_df.merge(df_rates, on='entry_id', suffixes=('', '_y'))

In [ ]:
crs = ccrs.EuroPP()
dem = xr.open_dataarray('../data/external/copdem_90m/copdem_90m_alps.tif')
dem = dem.isel(band=0)
dem = dem.rio.reproject(crs, resolution=500, resampling=rio.enums.Resampling.bilinear)

# set the bounds and convert to UTM
min_y, max_y, min_x, max_x = [44.5, 47.75, 5.75, 13.75]
min_x, min_y = crs.transform_point(min_x, min_y, ccrs.PlateCarree())
max_x, max_y = crs.transform_point(max_x, max_y, ccrs.PlateCarree())
min_x, min_y = int(min_x), int(min_y)
max_x, max_y = int(max_x), int(max_y)

# clip the DEM and compute the hillshade
dem = dem.rio.clip_box(min_x, min_y, max_x, max_y, crs=crs)
hillshade = es.hillshade(dem.values, azimuth=180, altitude=75)
hillshade = gaussian_filter(hillshade, sigma=0.5)

# make sure the bounds are not too small
gl_df_with_rates_proj = gl_df_with_rates.to_crs(crs)
assert min_x < gl_df_with_rates_proj.bounds.min().minx
assert max_x > gl_df_with_rates_proj.bounds.max().maxx
assert min_y < gl_df_with_rates_proj.bounds.min().miny
assert max_y > gl_df_with_rates_proj.bounds.max().maxy
fig_width = 10

fig, ax = plt.subplots(
    nrows=1,
    dpi=250,
    figsize=(fig_width, fig_width * (max_y - min_y) / (max_x - min_x)),
    subplot_kw={'projection': crs}
)

ext = [min_x, max_x, min_y, max_y]

# draw the hillshade (after darkening it)
hillshade = (hillshade - hillshade.min()) / (hillshade.max() - hillshade.min())
# hillshade = (hillshade / hillshade.max()) ** 3
# hillshade = hillshade * 0.5 + 0.5
hillshade = hillshade * 0.3
ax.imshow(hillshade, extent=ext, cmap='gray', zorder=-1, interpolation='nearest', alpha=0.9, vmin=0, vmax=1)

# draw the glacier outlines
# gl_df_with_rates_proj.simplify(100).plot(ax=ax, edgecolor='none', facecolor='#1434A4')
# gl_df_with_rates_proj.simplify(100).plot(ax=ax, edgecolor='none', facecolor='#0818A8', zorder=1)
gl_df_with_rates_proj.simplify(100).plot(ax=ax, edgecolor='none', facecolor='#87CEEB', zorder=1)

# add axis ticks
gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5, zorder=0, color='gray')
gl.zorder = 0

# add the country borders and the country names
ax.add_feature(cartopy.feature.BORDERS, zorder=0, linestyle='-', linewidth=0.75, alpha=1.0, color='orange')
country_coords = {
    'France': (6.25, 45.5),
    'Switzerland': (7.0, 46.5),
    'Austria': (11.5, 47.15),
    'Italy': (11.5, 46.5)
}
for cn, (x, y) in country_coords.items():
    ax.annotate(
        cn,
        xy=crs.transform_point(x, y, ccrs.PlateCarree()),
        xytext=(5, 5),
        textcoords='offset points',
        ha='center',
        va='bottom',
        fontsize=10,
        color='black',
        bbox=dict(boxstyle='round,pad=0.5', fc='white', ec='gray', lw=0.5)
    )

# add the scalebar
ax.add_artist(ScaleBar(dx=1.0, length_fraction=0.25, font_properties={'size': 10}, location='lower right'))
plt.tight_layout()

# export to pdf
plt.savefig(plots_dir / 'spatial_extent_glaciers.pdf', bbox_inches='tight')

plt.show()

In [ ]:
# group the glaciers into cells of 10km x 10km cells using the centroid
dx = 10e3

# use only the rates which were not filtered
gl_df_with_rates_ok = gl_df_with_rates[gl_df_with_rates.filtered == False].copy()

plot_0205 = False
if plot_0205:
    gl_df_with_rates_ok = gl_df_with_rates_ok[gl_df_with_rates_ok.area_class == '[0.2, 0.5)']

t = pyproj.Transformer.from_crs(gl_df_with_rates_ok.crs, crs, always_xy=True)
centroids_wgs84 = t.transform(gl_df_with_rates_ok.centroid.x, gl_df_with_rates_ok.centroid.y)
gl_df_with_rates_ok['lat'] = centroids_wgs84[0]
gl_df_with_rates_ok['lon'] = centroids_wgs84[1]

gl_df_with_rates_ok['lat_bin'] = (gl_df_with_rates_ok.lat // dx) * dx
gl_df_with_rates_ok['lon_bin'] = (gl_df_with_rates_ok.lon // dx) * dx
gdf_avg_s = gl_df_with_rates_ok.groupby(['lat_bin', 'lon_bin']).agg(
    {'area_rate': 'sum', 'entry_id': 'count', 'area_inv': 'sum'}).reset_index()
gdf_avg_s['area_rate_prc'] = gdf_avg_s.area_rate / gdf_avg_s.area_inv * 100
gdf_avg_s

In [ ]:
fig, axes = plt.subplots(
    nrows=2,
    dpi=250,
    figsize=(fig_width, fig_width * (max_y - min_y) / (max_x - min_x) * 1.03 + 0.1),
    gridspec_kw={"height_ratios": [1, 0.05], 'hspace': 0.15},
    subplot_kw={'projection': crs}
)

### 1) draw the glacier outlines
ax = axes[0]

# Apply gamma correction and rescale brightness
gamma = 10  # slightly brighten the mid-tones (lower is brighter)
hillshade_g = hillshade ** (1 / gamma)

# draw the hillshade (processed in the previous cell)
ax.imshow(hillshade_g, extent=ext, cmap='gray', zorder=-1, interpolation='nearest', alpha=0.9, vmin=0, vmax=1)

# add axis ticks
gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5, zorder=0, color='gray')
gl.zorder = 0

# keep only the cells with more than 1 km2 glacierized areas, to enhance the contrast
if plot_0205:
    tdf = gdf_avg_s[gdf_avg_s.area_inv >= 0.5].copy()
    area_factor = 30
    custom_sizes = [0.5, 1, 2]
else:
    area_factor = 2
    custom_sizes = [5, 25, 50, 100]
    tdf = gdf_avg_s[gdf_avg_s.area_inv >= 2].copy()

sc = ax.scatter(tdf.lat_bin, tdf.lon_bin, s=tdf.area_inv * area_factor, c=tdf.area_rate_prc, cmap='inferno', zorder=2)
fig.colorbar(
    sc, ax=axes[1], label='Annual area change rate (% $\\cdot y^{-1}$)', orientation="horizontal", fraction=0.9,
    pad=0.01
)

# define custom sizes for the legend
legend_elements = [
    matplotlib.lines.Line2D(
        [0], [0],
        marker='o',
        color='w',
        markerfacecolor='gray',
        markersize=np.sqrt(size * area_factor),
        label=f'{size}',
        linestyle='None'
    )
    for size in custom_sizes
]
ax.legend(handles=legend_elements, title="initial\narea\n($km^2$)", loc='lower right')

# add the country borders and the country names
ax.add_feature(cartopy.feature.BORDERS, zorder=0, linestyle='-', linewidth=0.5, alpha=0.75, color='orange')

# add the scalebar
ax.add_artist(ScaleBar(dx=1.0, length_fraction=0.25, font_properties={'size': 10}, location='upper left'))
plt.tight_layout()

# export to pdf
fn = 'area_change_rates_map_0205' if plot_0205 else 'area_change_rates_map'
plt.savefig(plots_dir / f'{fn}.pdf', bbox_inches='tight')

plt.show()

In [ ]:
plot_df = gl_df_with_rates_ok.copy()
plot_df.area_rate_prc = plot_df.area_rate_prc * 8  # use cumulative changes
plot_df['area_class'] = plot_df['area_class'].apply(lambda s: f"${s}$".replace(r'$\geq$', r'\geq'))

plot_df_text = plot_df.groupby('area_class', observed=True).agg(
    {'year_t0': 'count', 'area_inv': 'sum', 'area_rate_prc': 'min'}).reset_index()
plot_df_text['area_total_prc'] = plot_df_text.area_inv / df_rates.area_inv.sum() * 100
plot_df_text['y_pos'] = plot_df_text.area_rate_prc * 100 * 0.75
plot_df_text['desc_n'] = plot_df_text.apply(lambda r: f'n = {r.year_t0}', axis=1)
plot_df_text['desc_area'] = plot_df_text.apply(lambda r: rf'A_{{\%}} = {r.area_total_prc:.1f}\%', axis=1)

plot_df_text

In [ ]:
y_min = round(plot_df.area_rate_prc.min() * 100 * 1.2)
y_max = round(plot_df.area_rate_prc.max() * 100)
g = (
        p9.ggplot(plot_df, p9.aes(x='area_class', y='area_rate_prc * 100'))
        + p9.geom_boxplot()
        + p9.geom_text(plot_df_text, p9.aes(x='area_class', label='desc_n'), y=y_min * 0.875, va='top', size=10,
                       parse=True)
        + p9.geom_text(plot_df_text, p9.aes(x='area_class', label='desc_area'), y=y_min * 0.95, va='top', size=10,
                       parse=True)
        + p9.labs(
    y=r'2023-2015 relative area change (%)',
    x=r'Glacier size class (km$^2$)'
)
        + p9.ylim(y_min, y_max)
        + p9.theme(
    figure_size=(8, 4),
    dpi=200,
    legend_direction='horizontal',
    legend_position='bottom',
    text=p9.element_text(family='cmr10', size=12)
)
)
display(g)

fp = plots_dir / 'area_change_per_class.pdf'
fp.parent.mkdir(parents=True, exist_ok=True)
g.save(fp)


In [ ]:
plot_df = gl_df_with_rates_ok.rename(
    columns={'ELEV_MED': 'elev_med', 'ASP_MEAN': 'aspect', 'SLOPE_MEAN': 'slope'}).copy()
plot_df.area_rate_prc = plot_df.area_rate_prc * 8  # use cumulative changes

# var_name = 'elev_med'
# var_name = 'slope'
var_name = 'aspect'

if var_name in ('elev_med', 'slope'):
    if var_name == 'elev_med':
        x_label = 'Glacier median elevation (m)'
        steps = [2900, 3050, 3200]
    else:
        x_label = 'Slope (°)'
        steps = [18, 23, 28]
    all_groups = []
    for i in range(len(steps) + 1):
        if i == 0:
            idx = plot_df[var_name] < steps[i]
            crt_group = f"<{steps[i]}"
        elif i == len(steps):
            idx = plot_df[var_name] > steps[i - 1]
            crt_group = f">{steps[i - 1]}"
        else:
            idx = (plot_df[var_name] >= steps[i - 1]) & (plot_df[var_name] < steps[i])
            crt_group = f"{steps[i - 1]}\n-\n{steps[i]}"
        print(i, sum(idx))
        plot_df.loc[idx, 'group'] = crt_group
        if crt_group not in all_groups:
            all_groups.append(crt_group)
elif var_name == 'aspect':
    all_groups = ['N', 'E', 'S', 'W']
    x_label = 'Aspect'
    for v in all_groups:
        if v == 'N':
            idx = (plot_df.aspect <= 45) | (plot_df.aspect > 315)
        elif v == 'E':
            idx = (plot_df.aspect > 45) & (plot_df.aspect <= 135)
        elif v == 'S':
            idx = (plot_df.aspect > 135) & (plot_df.aspect <= 225)
        else:
            idx = (plot_df.aspect > 225) & (plot_df.aspect <= 315)
        plot_df.loc[idx, 'group'] = v

plot_df.group = pd.Categorical(plot_df.group, categories=all_groups)

# Calculate group means and round to 2 decimal places
mean_data = pd.DataFrame(plot_df.groupby('group', observed=True).area_rate_prc.mean() * 100).reset_index()
mean_data.columns = ['group', 'area_rate_prc']

# Prepare text annotations
plot_df_text = plot_df.groupby('group', observed=True).agg(
    {'year_t0': 'count', 'area_inv': 'sum', 'area_rate_prc': 'min'}).reset_index()
plot_df_text['area_total_prc'] = plot_df_text.area_inv / df_rates.area_inv.sum() * 100  # %A w.r.t. the total inv
plot_df_text['y_pos'] = plot_df_text.area_rate_prc * 100 * 0.75
plot_df_text['desc_n'] = plot_df_text.apply(lambda r: f'n = {r.year_t0}', axis=1)
plot_df_text['desc_area'] = plot_df_text.apply(lambda r: rf'A_{{\%}} = {r.area_total_prc:.1f}\%', axis=1)

# Calculate y-axis limits
y_min, y_max = -35, 0

# Create the plot
g = (
        p9.ggplot(mean_data, p9.aes(x='group', y='area_rate_prc')) +
        p9.geom_bar(stat='identity') +
        p9.geom_text(plot_df_text, p9.aes(x='group', label='desc_n'), y=y_min * 0.875, va='top', size=10, parse=True) +
        p9.geom_text(plot_df_text, p9.aes(x='group', label='desc_area'), y=y_min * 0.95, va='top', size=9,
                     parse=True) +
        p9.labs(
            y=r'2023-2015 relative area change (%)',
            x=x_label
        ) +
        p9.ylim(y_min, y_max) +
        p9.theme(
            figure_size=(3.5, 4),
            dpi=200,
            legend_direction='horizontal',
            legend_position='bottom',
            text=p9.element_text(family='sans-serif', size=12)
        )
)
display(g)

### Figure: overlays for both 2015 and 2023 (using the manually labelled glaciers)

In [ ]:
gl_df_inv = gpd.read_file(outlines_fp)

In [ ]:
entry_id = 'g_0498'
r_gl = gl_df_inv[gl_df_inv.entry_id == entry_id].iloc[0]
print(f"Glacier ID: {r_gl.entry_id.replace('g_', '')}\n({r_gl.Country} - {r_gl.LAT:.2f}° N, {r_gl.LON:.2f}° E)\n")

In [ ]:
# entry_id = 'g_0708'
# entry_id = 'g_0731'
# entry_id = 'g_0797'
# entry_id = 'g_1006'
entry_id = 'g_0954'

# read the raster data (for the inventory year and 2023)
year = 'inv'
# year = '2023'

fp_list = list(Path(f'../data/external/wd/s2_alps_plus/{year}/glacier_wide_sel/{entry_id}').glob('*.nc'))
assert len(fp_list) == 1
fp = fp_list[0]

nc = xr.open_dataset(fp, decode_coords='all')

# zoom in
# nc = nc.isel(x=slice(50, -50), y=slice(100, -100))
nc = nc.isel(x=slice(50, -50), y=slice(50, -50))

# read the ensemble predictions
if year == '2023':
    gl_df = gpd.read_file('../data/outlines/manual_delineation/preds_2023_corrected.shp')
    gl_df_pred = gpd.read_file(
        model_root_dir / 'gdf_all_splits/seed_all/version_0/s2_alps_plus/2023/2023_preds_calib.shp')
else:
    gl_df = gpd.read_file(outlines_fp)
    gl_df_pred = gpd.read_file(
        model_root_dir / 'gdf_all_splits/seed_all/version_0/s2_alps_plus/inv/inv_preds_calib.shp')

# reproject to the local UTM
gl_df_proj = gl_df.to_crs(nc.rio.crs)
gl_df_pred = gl_df_pred.to_crs(nc.rio.crs)

In [ ]:
# gl_df_proj[gl_df_proj.entry_id.isin(
#     ['g_0042', 'g_0042', 'g_0597', 'g_1660', 'g_0692', 'g_0708', 'g_0731', 'g_0791', 'g_0797', 'g_0491', 'g_1006'])]

In [ ]:
# plot the SWIR-NIR-R image
fig = plt.figure(dpi=300, figsize=(5, 5))
img1 = nc.band_data.isel(band=[nc.band_data.long_name.index(b) for b in ['SWIR', 'NIR', 'R']]).values.copy()
img1 = img1.transpose(1, 2, 0)

img1 = nc.band_data.isel(band=[nc.band_data.long_name.index(b) for b in ['R', 'G', 'B']]).values.copy()
img1 = img1.transpose(1, 2, 0)

# clip and scale the image
q = 0.03
vmin = np.quantile(img1, q)
vmax = np.quantile(img1, 1 - q)
img1 = np.clip(img1, vmin, vmax)
img1 = (img1 - vmin) / (vmax - vmin)

extent = [nc.x.min(), nc.x.max(), nc.y.min(), nc.y.max()]
plt.imshow(img1, extent=extent)
# plt.grid(alpha=0.1, linestyle='--')
plt.grid(False)
plt.xlabel('Easting [m]')
plt.ylabel('Northing [m]')

# plot the glacier outlines
gl_df_proj[gl_df_proj.entry_id == entry_id].plot(
    ax=plt.gca(), facecolor='none', edgecolor='C2', linewidth=1
)

# plot the ensemble prediction
gl_df_pred[gl_df_pred.entry_id == entry_id].plot(
    ax=plt.gca(), facecolor='none', edgecolor='C1', linewidth=1
)

show_labels_on_first_only = False

if not show_labels_on_first_only or entry_id == 'g_0708':
    # build the legend (manually; default fails with UserWarning: Legend does not support handles for PatchCollection instances)
    # y = gl_df_proj[gl_df_proj.entry_id == entry_id].date_inv.iloc[0].year
    y = fp.stem[:4]
    legend_handles = [
        matplotlib.lines.Line2D([0], [0], color='C2',
                                label=f'Reference ({y})' if year == '2023' else f'Inventory ({y})'),
        matplotlib.lines.Line2D([0], [0], color='C1', label=f'Prediction ({y})'),
    ]
    # Add the custom legend to the plot
    plt.legend(handles=legend_handles, loc='upper right')

r_gl = gl_df_inv[gl_df_inv.entry_id == entry_id].iloc[0]
print(f"Glacier ID: {r_gl.entry_id.replace('g_', '')}\n({r_gl.Country} - {r_gl.LAT:.2f}° N, {r_gl.LON:.2f}° E)\n")

plt.gca().add_artist(
    ScaleBar(
        dx=1.0,
        length_fraction=0.2,
        font_properties={'size': 12},
        frameon=True,
        scale_loc='right',
        location='lower right'
    )
)

# disable the axis labels
plt.axis('off')

# plt.savefig(plots_dir / f"{entry_id}_{year}.pdf", bbox_inches='tight', dpi=300)

plt.show()

### Figure: overlays for 2015 for the SGI glaciers used in the round-robin experiment

In [ ]:
entry_id_sel = [
    'B35-02',
    'A12d-10',
    'E35-19',
    'B51-12',
    'B72-08',
    'A54e-13',
    'B90-04',
    'C05-02',
    'B22-01',
    'A51f-10',
    'E23-06',
    'B63-05',
    'A54g-11'
]

In [ ]:
# read the raster data (for the inventory year and 2023)
year = 'inv'

for entry_id in sorted(entry_id_sel):
    fp_list = list(Path(f'../data/external/wd/s2_sgi/{year}/glacier_wide_sel/{entry_id}').glob('*.nc'))
    assert len(fp_list) == 1
    fp = fp_list[0]

    nc = xr.open_dataset(fp, decode_coords='all')

    # zoom in
    if entry_id == 'A12d-10':
        nc = nc.isel(x=slice(50, -50), y=slice(50, -50))
    else:
        nc = nc.isel(x=slice(50, -50), y=slice(100, -100))

    # read the ensemble predictions
    gl_df_inv = gpd.read_file('../data/outlines/sgi2016/SGI_2016_glaciers.shp')
    gl_df = gpd.read_file('../data/outlines/sgi2016/SGI_2016_glaciers.shp')
    gl_df_pred = gpd.read_file(model_root_dir / 'gdf_all_splits/seed_all/version_0/s2_sgi/inv/inv_preds_calib.shp')

    # reproject to the local UTM
    gl_df_proj = gl_df.to_crs(nc.rio.crs)
    gl_df_pred = gl_df_pred.to_crs(nc.rio.crs)

    # plot the SWIR-NIR-R image
    fig = plt.figure(dpi=300, figsize=(5, 5))
    img1 = nc.band_data.isel(band=[nc.band_data.long_name.index(b) for b in ['SWIR', 'NIR', 'R']]).values.copy()
    img1 = img1.transpose(1, 2, 0)

    # clip and scale the image
    q = 0.03
    vmin = np.quantile(img1, q)
    vmax = np.quantile(img1, 1 - q)
    img1 = np.clip(img1, vmin, vmax)
    img1 = (img1 - vmin) / (vmax - vmin)

    extent = [nc.x.min(), nc.x.max(), nc.y.min(), nc.y.max()]
    plt.imshow(img1, extent=extent)
    # plt.grid(alpha=0.1, linestyle='--')
    plt.grid(False)
    plt.xlabel('Easting [m]')
    plt.ylabel('Northing [m]')

    # plot the glacier outlines
    gl_df_proj[gl_df_proj.entry_id == entry_id].plot(
        ax=plt.gca(), facecolor='none', edgecolor='C2', linewidth=1
    )

    # plot the ensemble prediction
    gl_df_pred[gl_df_pred.entry_id == entry_id].plot(
        ax=plt.gca(), facecolor='none', edgecolor='C1', linewidth=1
    )

    if entry_id == 'A12d-10':
        # build the legend (manually; default fails with UserWarning: Legend does not support handles for PatchCollection instances)
        # y = gl_df_proj[gl_df_proj.entry_id == entry_id].date_inv.iloc[0].year
        y = fp.stem[:4]
        legend_handles = [
            matplotlib.lines.Line2D([0], [0], color='C2', label=f'Inventory (SGI2016)'),
            matplotlib.lines.Line2D([0], [0], color='C1', label=f'DL4GAM Prediction'),
        ]
        # Add the custom legend to the plot
        plt.legend(handles=legend_handles, loc='upper right')

    r_gl = gl_df_inv[gl_df_inv.entry_id == entry_id].iloc[0]
    # print(f"Glacier ID: {r_gl.entry_id.replace('g_', '')}\n({r_gl.Country} - {r_gl.LAT:.2f}° N, {r_gl.LON:.2f}° E)\n")

    plt.gca().add_artist(
        ScaleBar(
            dx=1.0,
            length_fraction=0.2,
            font_properties={'size': 12},
            frameon=True,
            scale_loc='right',
            location='lower right'
        )
    )

    # disable the axis labels
    plt.axis('off')

    fp_out = plots_dir / f"sgi_{entry_id}_{year}.pdf"
    plt.savefig(fp_out, bbox_inches='tight', dpi=300)
    print(f"Plot saved to {fp_out}")

    # plt.show()
    # break